# Non Negative Matrix Factorization for Recommender Systems

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from movie_dataset import preprocess_movies
from movie_dataset import DISNEY_MOVIE_IDS, DISNEY_RELEVANT_HITS
from sklearn.decomposition import NMF
import pickle

In [ ]:
ratings, movies, R = preprocess_movies()

In [ ]:
ratings.shape, movies.shape

In [ ]:
ratings.head()

In [ ]:
ratings['userId'].nunique()

In [ ]:
movies.head()

In [ ]:
movies.loc[DISNEY_MOVIE_IDS]

In [ ]:
# users vs. movies combinations (huge shape because of high movie IDs)
R

### Train the Model

- initialize the model
- fit it on the user item matrix
- optionally, tune the number of components (hidden features)
- decrease the `tol` hyperparameter to train for a longer time

In [ ]:
model = NMF(n_components=55, init='nndsvd', max_iter=1000, tol=0.01, verbose=2)

model.fit(R)

# initialize P, Q matrix with random values
# iterate and optimize the values stored in P and Q

### Inspect the hidden features

In [ ]:
# P is a user-genre matrix. It describes what types of movies a given user prefers
P = model.transform(R)
P.shape

In [ ]:
P[2, ] # user no.2

In [ ]:
# Q represents how strongly each rating maps to any of the 55 components
Q = model.components_
Q.shape

### Calculate Reconstruction Error

$$
L(R, \hat{R}) = \sqrt{\sum_i\sum_j(R_{ij}-\hat{R}_{ij})^2} = \sqrt{\sum_i\sum_j(R_{ij}-PQ_{ij})^2}
$$

In [ ]:
# R -> encoding -> P -> decoding -> Rhat
R_hat = model.inverse_transform(P)

In [ ]:
# compare reconstruction to manually calculated one
print(model.reconstruction_err_)

# MATRIX FACTORIZATON: R=PQ
R_hat = P.dot(Q)

np.sqrt(np.sum(np.square(R - R_hat)))

### Make recommendations

In [ ]:
DISNEY_MOVIE_IDS

### Construct a user vector

we need the same input as was used during training!

In [ ]:
# new user vector: needs to have the same format as the training data
# pre fill it with zeros
user_vec = np.zeros(168253)

# fill in the ratings that arrived from the query
user_vec[DISNEY_MOVIE_IDS] = 5
user_vec

### Calculate the score

1. transform the user vector to its dense representation (encoding) 
2. inverse transform the dense vector into the sparse representation (decoding)

$$
\hat{r}_{ij} = p_i' \cdot q_j 
$$

In [ ]:
encoding = model.transform([user_vec])  # strength of the 55 hidden component
encoding

In [ ]:
scores = model.inverse_transform(encoding)  # (1, 168253)
scores = pd.Series(scores[0])
scores

### Give recommendations

In [ ]:
# give a zero score to movies the user has allready seen
scores[DISNEY_MOVIE_IDS] = 0

In [ ]:
# sort the scores from high to low 
scores = scores.sort_values(ascending=False)
scores

In [ ]:
# get the movieIds of the top 10 entries
recommendations = scores.head(10).index
recommendations

In [ ]:
movies.loc[recommendations]

In [ ]:
movies.loc[DISNEY_RELEVANT_HITS]

In [ ]:
# recall@10: fraction of rele'vant items in the top 10 recommendations
pd.Series(DISNEY_RELEVANT_HITS).isin(recommendations).mean()

In [ ]:
# precision@10: fraction of recommendations that are relevant
recommendations.isin(DISNEY_RELEVANT_HITS).mean()

#### Save the trained model

In [ ]:
with open('./nmf_recommender.pkl', 'wb') as file:
    pickle.dump(model, file)

#### Load the trained model

In [ ]:
with open('./nmf_recommender.pkl', 'rb') as file:
    model = pickle.load(file)